# pandas怎么实现DataFrame的Merge
pandasdeMerge,想打渔Sql的Join, 将不同的表按key关联到一个表

## merge的语法:
pd.merge(left,right,how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=True,
suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

- left, right:要merge的DataFrame或者有name的Series
- how:join类型, 'left', 'right', 'outer', 'inner'
- on: join的key, left和right都需要这个key
- left_on: left的DataFrame或Series的可以
- right_on: right的DataFrame或Series的key
- left_index, right_index: 使用index二不是普通的column做join
- suffixes: 两个元素的后缀, 如果列有重名, 自动添加后缀, 默认是('_x', '_y')

[文档地址](hitps://pandas,pydata org/pandas-docs/stablelreferencelaplpandas.DataFrame.merge.html)

本次示例题纲:
1. 电影数据集的join示例
2. 理解merge是一对一, 一对多, 多对多的数量对齐关系
3. 理解left join、 right join、 inner join、 outer join的区别
4. 如果出现非Key的字段重名怎么办

## 1. 电影数据集的join实例

In [11]:
import pandas as pd

In [17]:
# engine=python参数用于在分割符号为两个重复符号时, pandas会默认认为是正则表达式, 需要指定一下
# names=[] 当数据集没有抬头行, 也就是字段名称时需要指定
df_ratings = pd.read_csv('./data/movies/ratings.csv', sep=',', engine='python')

In [18]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
df_user = pd.read_excel('./data/movies/users.xlsx')

In [20]:
df_user.head()

,userId,age,gender,occupation,zip-code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [26]:
df_movies = pd.read_csv('./data/movies/movies.csv', sep=',')

In [28]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [30]:
# merge第一种方式
df_ratings_user = pd.merge(df_ratings, df_user, left_on='userId', right_on='userId', how='inner')

In [32]:
df_ratings_user.head()

,userId,movieId,rating,timestamp,age,gender,occupation,zip-code
0,1,1,4.0,964982703,24,M,technician,85711
1,1,3,4.0,964981247,24,M,technician,85711
2,1,6,4.0,964982224,24,M,technician,85711
3,1,47,5.0,964983815,24,M,technician,85711
4,1,50,5.0,964982931,24,M,technician,85711


In [46]:
# merge第二种方式
df_ratings.merge(df_user, on='userId', how='inner')

,userId,movieId,rating,timestamp,age,gender,occupation,zip-code
0,1,1,4.0,964982703,24,M,technician,85711
1,1,3,4.0,964981247,24,M,technician,85711
2,1,6,4.0,964982224,24,M,technician,85711
3,1,47,5.0,964983815,24,M,technician,85711
4,1,50,5.0,964982931,24,M,technician,85711
...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,22,M,student,21227
100832,610,168248,5.0,1493850091,22,M,student,21227
100833,610,168250,5.0,1494273047,22,M,student,21227
100834,610,168252,5.0,1493846352,22,M,student,21227


In [47]:
df_ratings_user_movies = pd.merge(df_ratings_user, df_movies, left_on='movieId', right_on='movieId', how='inner')

In [49]:
df_ratings_user_movies.head()

,userId,movieId,rating,timestamp,age,gender,occupation,zip-code,title,genres
0,1,1,4.0,964982703,24,M,technician,85711,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,33,F,other,15213,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,57,M,administrator,91344,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,49,F,educator,97301,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,30,M,programmer,06355,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


## 2.理解merge时数量对齐关系
以下关系要正确理解:
+ one-to-one: 一对一关系， 关联的key都是唯一的
  + 比如(学号, 姓名)merge(学号, 年龄)
  + 结果条数为： 1*1
+ one_to_many: 一对多关系， 左边唯一key, 右边不唯一key
    + 比如(学号, 姓名)merge(学号, [语文成绩、数学成绩、英语成绩])
    + 结果条数1*N
+ many-to-many: 多对多关系, 左边右边都不是唯一
    + 比如(学号, [语文成绩、数学成绩、英语成绩])merge(学号, [篮球, 足球, 乒乓球])
    + 结果条数为: M*N

### 2.1 one-to-one
<img src='./image/one_to_one.png' />

In [52]:
left = pd.DataFrame({'sno': [11, 12, 13, 14],
                     'name': ['name_a', 'name_b', 'name_c', 'name_d']})
left

,sno,name
0,11,name_a
1,12,name_b
2,13,name_c
3,14,name_d


In [53]:
right = pd.DataFrame({'sno': [11, 12, 13, 14],
                      'age': ['21', '22', '23', '24']})
right

,sno,age
0,11,21
1,12,22
2,13,23
3,14,24


In [56]:
# 一对一关系, 结果种有4条
pd.merge(left=left, right=right, on='sno')

,sno,name,age
0,11,name_a,21
1,12,name_b,22
2,13,name_c,23
3,14,name_d,24


### 2.2 one-to-many 一对多关系的merge
注意: 数据会被复制
<img src="./image/one_to_many.png">


In [57]:
left1 = pd.DataFrame({'sno': [11, 12, 13, 14],
                      'name': ['name_a', 'name_b', 'name_c', 'name_d']})
left1

,sno,name
0,11,name_a
1,12,name_b
2,13,name_c
3,14,name_d


In [61]:
right1 = pd.DataFrame({'sno': [11, 11, 11, 12, 12, 13],
                       'grade': ['语文88', '数学90', '英语75', '语文66', '数学55', '英语29']})
right1

,sno,grade
0,11,语文88
1,11,数学90
2,11,英语75
3,12,语文66
4,12,数学55
5,13,英语29


In [67]:
pd.merge(left=left1, right=right1, on='sno')

,sno,name,grade
0,11,name_a,语文88
1,11,name_a,数学90
2,11,name_a,英语75
3,12,name_b,语文66
4,12,name_b,数学55
5,13,name_c,英语29


### 2.3 many-to-many 多对多关系
注意: 结果会出现乘法
<img src="./image/many_to_many.png">


In [64]:
left2 = pd.DataFrame({'sno': [11, 11, 12, 12, 12],
                      '爱好': ['篮球', '羽毛球', '乒乓球', '篮球', '足球']})
left2

,sno,爱好
0,11,篮球
1,11,羽毛球
2,12,乒乓球
3,12,篮球
4,12,足球


In [65]:
right2 = pd.DataFrame({'sno': [11, 11, 11, 12, 12, 13],
                       'grade': ['语文88', '数学90', '英语75', '语文66', '数学55', '英语29']})
right2

,sno,grade
0,11,语文88
1,11,数学90
2,11,英语75
3,12,语文66
4,12,数学55
5,13,英语29


In [66]:
pd.merge(left=left2, right=right2, on='sno')

,sno,爱好,grade
0,11,篮球,语文88
1,11,篮球,数学90
2,11,篮球,英语75
3,11,羽毛球,语文88
4,11,羽毛球,数学90
5,11,羽毛球,英语75
6,12,乒乓球,语文66
7,12,乒乓球,数学55
8,12,篮球,语文66
9,12,篮球,数学55


## 3. left join、 right join、 inner join、 outer join的区别
<img src="./image/left_join-right_join-inner_join-outer_join的区别.png">

In [68]:
left3 = pd.DataFrame({'key':['K0', 'K1', 'K2', 'K3'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0','B1', 'B2', 'B3']})
right3 = pd.DataFrame({'key':['K0', 'K1', 'K4', 'K5'],
                       'C': ['C0', 'C1', 'C4', 'C5'],
                       'D': ['D0', 'D1', 'D4', 'D5']})

In [69]:
left3

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [70]:
right3

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K4,C4,D4
3,K5,C5,D5


### 3.1 inner join, 默认
左边和右边的key都有才会出现在结果里

In [73]:
pd.merge(left=left3, right=right3, how='inner', on='key')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1


### 3.2 left join
左边的都会出现在结果里， 右边的如果无法匹配则为Null

In [75]:
pd.merge(left=left3, right=right3, on='key', how='left')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,NaN,NaN
3,K3,A3,B3,NaN,NaN


### 3.3 right join
右边的都会出现在结果里, 左边的如果无法匹配则为Null

In [77]:
pd.merge(left=left3, right=right3, on='key', how='right')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K4,NaN,NaN,C4,D4
3,K5,NaN,NaN,C5,D5


### 3.4 outer join
左边、右边的都会出现在结果里, 如果无法匹配则为Null

In [78]:
pd.merge(left=left3, right=right3, on='key', how='outer')

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,NaN,NaN
3,K3,A3,B3,NaN,NaN
4,K4,NaN,NaN,C4,D4
5,K5,NaN,NaN,C5,D5


## 4. 如果出现非key的字段重名怎么办

In [80]:
left4 = pd.DataFrame({'key':['K0', 'K1', 'K2', 'K3'],
                      'A': ['A0', 'A1', 'A2', 'A3'],
                      'B': ['B0','B1', 'B2', 'B3']})

right4 = pd.DataFrame({'key': ['K0', 'K1', 'K4', 'K5'],
                       'A': ['A10', 'A11', 'A12', 'A13'],
                       'D': ['D0', 'D1', 'D4', 'D5']})

In [82]:
left4

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [83]:
right4

,key,A,D
0,K0,A10,D0
1,K1,A11,D1
2,K4,A12,D4
3,K5,A13,D5


In [84]:
pd.merge(left4, right4, on='key')

,key,A_x,B,A_y,D
0,K0,A0,B0,A10,D0
1,K1,A1,B1,A11,D1


In [86]:
pd.merge(left4, right4, on='key', suffixes=('_left', '_right'))

,key,A_left,B,A_right,D
0,K0,A0,B0,A10,D0
1,K1,A1,B1,A11,D1
